In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline



In [2]:
import requests
webseitdata=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


In [3]:
# parse data from the html into a beautifulsoup object
from bs4 import BeautifulSoup
soup = BeautifulSoup(webseitdata,'html.parser')
#print(soup.prettify())


In [4]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []


In [5]:
#The table is available at class
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text)


In [6]:
postalCodeList=list(map(lambda x:x.rstrip("\n"),postalCodeList))
boroughList=list(map(lambda x:x.rstrip("\n"),boroughList))
neighborhoodList=list(map(lambda x:x.rstrip("\n"),neighborhoodList))


In [7]:
df=pd.DataFrame({"PostalCode":postalCodeList, "Borough":boroughList,"Neighborhood":neighborhoodList})


In [8]:
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_clean1=df[df.Borough!="Not assigned"].reset_index(drop=True)


In [10]:
df_clean1.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# group neighborhoods in the same borough
toronto_df_grouped = df_clean1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
toronto_df_grouped["Neighborhood"]=toronto_df_grouped["Neighborhood"].replace("Not assigned",toronto_df_grouped["Borough"])


In [13]:
toronto_df_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

test_df=pd.DataFrame(columns=toronto_df_grouped.columns)

for postalcode in test_list:
    test_df=test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postalcode],ignore_index=True)


In [15]:
test_df


,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [16]:
toronto_df_grouped.shape


(103, 3)

In [17]:
geo_url="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(geo_url)


In [18]:
#geo_data.columns
geo_data.columns=['PostalCode', 'Latitude', 'Longitude']


In [19]:
toronto_df2=pd.merge(toronto_df_grouped,geo_data,how='inner',on="PostalCode")


In [20]:
toronto_df2


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df2['Borough'].unique()),
        toronto_df2.shape[0]
    )
)


The dataframe has 10 boroughs and 103 neighborhoods.


In [22]:
from geopy.geocoders import Nominatim 
import geopy
# convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [23]:
#get Latitute and longitude of toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of toronto City are 43.6534817, -79.3839347.


In [24]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df2['Latitude'], toronto_df2['Longitude'], toronto_df2['Borough'], toronto_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto


In [25]:
df_borough_toronto=toronto_df2[toronto_df2["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_borough_toronto.size


195

In [26]:
df_borough_toronto.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
df_borough_toronto["Borough"].unique()


array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [28]:
df_borough_toronto["color"]=df_borough_toronto["Borough"].map({'East Toronto':"green", 'Central Toronto':"red", 'Downtown Toronto':"blue",
       'West Toronto':"black"})


In [29]:
df_borough_toronto.shape


(39, 6)

In [30]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood,clr in zip(df_borough_toronto['Latitude'], df_borough_toronto['Longitude'], df_borough_toronto['Borough'], df_borough_toronto['Neighborhood'],df_borough_toronto["color"]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=clr,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto


In [31]:
CLIENT_ID = '0GPNYOVQQJBQDIIH3TCFRMKBQKP5ZNKBH5CB4LI3VMD51442' # your Foursquare ID
CLIENT_SECRET = '33LVENI2BSLMN0OWWDSYCSMMAOWLZHAUXA1D4KTXYPI0A2HR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 0GPNYOVQQJBQDIIH3TCFRMKBQKP5ZNKBH5CB4LI3VMD51442
CLIENT_SECRET:33LVENI2BSLMN0OWWDSYCSMMAOWLZHAUXA1D4KTXYPI0A2HR


In [32]:
#first neigbourhood
neighborhood_latitude1=df_borough_toronto["Latitude"][0]
neighborhood_longitude1=df_borough_toronto["Longitude"][0]
neighborhood_name1=df_borough_toronto["Neighborhood"][0]

print (f"{neighborhood_name1} has lognitude and latitude as : [{neighborhood_latitude1},{neighborhood_longitude1}]")


The Beaches has lognitude and latitude as : [43.67635739999999,-79.2930312]


In [33]:
# Setup API URL to explore venues near by
LIMIT=100
RADIUS=500
url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={neighborhood_latitude1},{neighborhood_longitude1}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]


In [34]:
# Serializing json
import json
json_object = json.dumps(neighborhood_json, indent = 4)


In [35]:
#save data as json file to explore
with open("jsonData.json","w") as f:
    f.write(json_object)


In [36]:
venues=neighborhood_json


In [37]:
#flatten Json
from pandas.io.json import json_normalize
nearby_venues=json_normalize(venues)


In [38]:
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']


In [39]:
nearby_venues=nearby_venues.loc[:,filtered_columns]


In [40]:
def getCategory_type(row):
    try:
        category_list=row["name"]
    except:
        category_list=row["venue.categories"]
    if len(category_list)==0:
        return None
    else:
        return category_list[0]["name"]


In [41]:
nearby_venues["categories"]= [x[0]["name"] for x in nearby_venues["venue.categories"]]


In [42]:
nearby_venues.drop(["venue.categories"],axis=1,inplace=True)


In [43]:
nearby_venues


,venue.name,venue.location.lat,venue.location.lng,categories
0,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
5,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [44]:
#we have just explored one neighbourhood and its nearby venues and their category now we will be doing for all nerwighborhood in
df_borough_toronto


,PostalCode,Borough,Neighborhood,Latitude,Longitude,color
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,green
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,green
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,green
3,M4M,East Toronto,Studio District,43.659526,-79.340923,green
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,red
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,red
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,red
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,red
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,red
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,red


In [45]:
CLIENT_ID = 'X5STHTOJ5ALK42QMTYDXRXWTKQ20FB4OOWKA5IQZ4PE1D5ZI' # your Foursquare ID
CLIENT_SECRET = 'VPEBQYOI35SAJ5EOP5L34DXJTRQIKHNT5ERK051GFSSQKNWU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
# Setup API URL to explore venues near by
LIMIT=100
RADIUS=500

def getNearByVenues(neighbourhood_name,lat,long):
    venues_list=[]

    for name, lat, lng in zip(neighbourhood_name,lat,long):
        print(name)
        
        url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{lng}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
        neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in neighborhood_json])
        #appending list of  venuedetails as list into another list venues list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    return (nearby_venues)


In [46]:
#Get all Tor
toronto_venues_df = getNearByVenues(df_borough_toronto['Neighborhood'],df_borough_toronto['Latitude'],df_borough_toronto['Longitude'])


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [47]:
toronto_venues_df.shape
#toronto_venues_df.head()


(1638, 7)

In [48]:
toronto_venues_df.groupby("Neighborhood").count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [49]:
#analyze the neighbourhoood
#creating dummy for each venue category

torento_onehot=pd.get_dummies(toronto_venues_df[["Venue Category"]], prefix="", prefix_sep="")


In [50]:
torento_onehot.shape


(1638, 232)

In [51]:
torento_onehot["Neighborhood"]=toronto_venues_df["Neighborhood"]


In [52]:
torento_onehot.head()


,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
torento_onehot.columns.get_loc("Neighborhood")


161

In [54]:
torento_onehot.columns[161]


'Neighborhood'

In [55]:
fixed_columns=[torento_onehot.columns[159]]+list(torento_onehot.columns[0:159])+list(torento_onehot.columns[160:])


In [56]:
len(fixed_columns)


232

In [57]:
torento_onehot=torento_onehot[fixed_columns]


In [58]:
torento_onehot.head()


,Museum,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
torento_onehot_grouped=torento_onehot.groupby("Neighborhood").mean().reset_index()


In [60]:
torento_onehot_grouped


,Neighborhood,Museum,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.017241,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.0000,0.00,0.015625
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
6,Church and Wellesley,0.000000,0.013158,0.000000,0.000000,0.000000,0.0,0.000000,0.013158,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.026316
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.00,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000


In [61]:
for  hood in torento_onehot_grouped["Neighborhood"]:
    print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    print(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10))
    dict1={}
    print("\n")


-------Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1              Bakery  0.03
2  Seafood Restaurant  0.03
3        Cocktail Bar  0.03
4      Farmers Market  0.03
5            Beer Bar  0.03
6                Café  0.03
7          Restaurant  0.03
8         Cheese Shop  0.03
9       Shopping Mall  0.02


-------Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1         Breakfast Spot  0.09
2  Performing Arts Venue  0.09
3            Coffee Shop  0.09
4      Convenience Store  0.04
5             Restaurant  0.04
6          Burrito Place  0.04
7                Stadium  0.04
8              Nightclub  0.04
9                    Bar  0.04


-------Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Comic Shop  0.06
4         Pizza Place  0.06
5   

                         venue  freq
0               Breakfast Spot  0.14
1                    Gift Shop  0.14
2                    Bookstore  0.07
3                      Dog Run  0.07
4                          Bar  0.07
5           Italian Restaurant  0.07
6                 Dessert Shop  0.07
7                   Restaurant  0.07
8  Eastern European Restaurant  0.07
9                Movie Theater  0.07


-------Queen's Park, Ontario Provincial Government----
                 venue  freq
0          Coffee Shop  0.30
1                Diner  0.07
2          Yoga Studio  0.03
3                 Bank  0.03
4                 Park  0.03
5   College Auditorium  0.03
6  Distribution Center  0.03
7       Sandwich Place  0.03
8                 Café  0.03
9        Burrito Place  0.03


-------Regent Park, Harbourfront----
                venue  freq
0         Coffee Shop  0.17
1                 Pub  0.07
2                Park  0.07
3              Bakery  0.07
4                Café  0.04
5         

In [62]:
dict1={}

for  hood in torento_onehot_grouped["Neighborhood"]:
    val=[]
    #print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    val=list(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10)["venue"])
    dict1[hood]=val


In [63]:
cols=["No."+str(x)+"_common_Place" for x in range(1,11)]


In [64]:
neighborhoods_venues_sorted=pd.DataFrame(dict1).T


In [65]:
neighborhoods_venues_sorted.columns=cols


In [66]:
neighborhoods_venues_sorted.insert(0,"Neighborhood",list(neighborhoods_venues_sorted.index))


In [67]:
neighborhoods_venues_sorted.reset_index(drop=True,inplace=True)


In [68]:
neighborhoods_venues_sorted.head()


,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,Berczy Park,Coffee Shop,Bakery,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Café,Restaurant,Cheese Shop,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Convenience Store,Restaurant,Burrito Place,Stadium,Nightclub,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Plane,Sculpture Garden,Coffee Shop,Rental Car Location,Boat or Ferry,Bar,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Department Store,Japanese Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Office


In [69]:
torento_onehot_grouped.head()


,Neighborhood,Museum,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.017241,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.066667,0.066667,0.133333,0.2,0.066667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.0,0.015625


In [70]:
#Training
from  sklearn.cluster import KMeans

#set no of clusters
n_cluster=5
#set gtraining Data
training_Data=torento_onehot_grouped.drop("Neighborhood",axis=1)
#Training the model
cluster_kmean=KMeans(n_clusters=n_cluster,random_state=0).fit(training_Data)
cluster_kmean


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [71]:
#check the labels
cluster_kmean.labels_


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [72]:
#adding cluster into venues tables
neighborhoods_venues_sorted.insert(0,"cluster_lablel",cluster_kmean.labels_)


In [73]:
neighborhoods_venues_sorted


,cluster_lablel,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,0,Berczy Park,Coffee Shop,Bakery,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Café,Restaurant,Cheese Shop,Shopping Mall
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Convenience Store,Restaurant,Burrito Place,Stadium,Nightclub,Bar
2,0,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Plane,Sculpture Garden,Coffee Shop,Rental Car Location,Boat or Ferry,Bar,Airport Terminal
4,0,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Department Store,Japanese Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Office
5,0,Christie,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Candy Store,Diner,Baby Store,Coffee Shop,Nightclub
6,0,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Men's Store,Dance Studio,Hotel
7,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Deli / Bodega,Asian Restaurant
8,0,Davisville,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Brewery
9,0,Davisville North,Gym,Hotel,Park,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,Gym / Fitness Center,Hobby Shop


In [74]:
torento_merged=toronto_df2.copy()


In [75]:
torento_merged=pd.merge(torento_merged,neighborhoods_venues_sorted,on="Neighborhood")


In [76]:
torento_merged.set_index("PostalCode",drop=True,inplace=True)


In [77]:
torento_merged

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Coffee Shop,Asian Restaurant,Pub,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Fruit & Vegetable Store,Frozen Yogurt Shop
M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Gym,Burrito Place,Food & Drink Shop,Light Rail Station,Board Shop,Fish & Chips Shop,Fast Food Restaurant,Brewery,Sandwich Place,Sushi Restaurant
M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Gym / Fitness Center,Bank,Seafood Restaurant,Fish Market
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Dim Sum Restaurant,Park,Swim School,Bus Line,Museum,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant
M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Hotel,Park,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,Gym / Fitness Center,Hobby Shop
M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Yoga Studio,Rental Car Location,Fast Food Restaurant,Pet Store,Spa,Sporting Goods Shop,Café,Salon / Barbershop
M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Brewery
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Trail,Restaurant,Museum,Moroccan Restaurant,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [78]:
torento_merged.columns


Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'cluster_lablel',
       'No.1_common_Place', 'No.2_common_Place', 'No.3_common_Place',
       'No.4_common_Place', 'No.5_common_Place', 'No.6_common_Place',
       'No.7_common_Place', 'No.8_common_Place', 'No.9_common_Place',
       'No.10_common_Place'],
      dtype='object')

In [79]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[torento_merged["Latitude"][0], torento_merged["Longitude"][0]], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_cluster)
ys = [i + x + (i*x)**2 for i in range(n_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng,neighborhood,cluster_label in zip(torento_merged['Latitude'], torento_merged['Longitude'], torento_merged['Neighborhood'],torento_merged["cluster_lablel"]):
    
    label = folium.Popup(str(neighborhood)+"cluster\n"+str(cluster_label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster_label],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto


In [80]:
torento_merged[torento_merged["cluster_lablel"]== 0]


,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Coffee Shop,Asian Restaurant,Pub,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Fruit & Vegetable Store,Frozen Yogurt Shop
M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Gym,Burrito Place,Food & Drink Shop,Light Rail Station,Board Shop,Fish & Chips Shop,Fast Food Restaurant,Brewery,Sandwich Place,Sushi Restaurant
M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Gym / Fitness Center,Bank,Seafood Restaurant,Fish Market
M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Hotel,Park,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,Gym / Fitness Center,Hobby Shop
M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Yoga Studio,Rental Car Location,Fast Food Restaurant,Pet Store,Spa,Sporting Goods Shop,Café,Salon / Barbershop
M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Brewery
M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,0,Pub,Coffee Shop,Bagel Shop,Light Rail Station,Sports Bar,Supermarket,Sushi Restaurant,Fried Chicken Joint,Bank,Restaurant
M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Café,Park,Pub,Italian Restaurant,Restaurant,Bakery,Pizza Place,Chinese Restaurant,Market


In [81]:
torento_merged[torento_merged["cluster_lablel"]== 1]


,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316,1,Trail,Restaurant,Museum,Moroccan Restaurant,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [82]:
torento_merged[torento_merged["cluster_lablel"]== 2]


,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Museum,Moroccan Restaurant,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store
M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Museum,Monument / Landmark,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant


In [83]:
torento_merged[torento_merged["cluster_lablel"]== 3]


,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Music Venue,Home Service,Garden,Movie Theater,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [84]:
torento_merged[torento_merged["cluster_lablel"]== 4]


,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,4,Dim Sum Restaurant,Park,Swim School,Bus Line,Museum,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant
